In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf

print("Tensorflow version:", tf.__version__)

# Check if TensorFlow is using a GPU
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is running on GPU")
else:
    print("TensorFlow is running on CPU")

In [ ]:
 #Load dataset

data_path = "dataset/"
emotions = os.listdir(data_path)

# Check for '.DS_Store' and delete it
if '.DS_Store' in emotions:
    os.remove(os.path.join(data_path, '.DS_Store'))
    emotions.remove('.DS_Store')

print(emotions)

In [ ]:
img_size = 48 #By default, the images in FER2013 dataset is in 48x48

data = []
labels = []

# Looping through subfolders in the data path
for emotion in emotions:
    emotion_folder = os.path.join(data_path, emotion)
    print(emotion_folder)
    if not os.path.isdir(emotion_folder):
        continue
    for img in os.listdir(emotion_folder):
        img_path = os.path.join(emotion_folder, img)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (img_size, img_size))
            data.append(img)
            labels.append(emotions.index(emotion))

# Convert image data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Plot the distribution of the labels, bar graph
plt.hist(labels, bins=len(emotions))
plt.xticks(range(len(emotions)), emotions, rotation=45)
plt.show()

In [ ]:
# def preprocess_images(images):
#     processed_images = []
#     for img in images:
#         # Convert to color if necessary
#         if img.shape[-1] != 3:
#             img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
#         # Resize the image to 48x48 if necessary
#         if img.shape[:2] != (48, 48):
#             img = cv2.resize(img, (48, 48))
        
#         processed_images.append(img)
    
#     return np.array(processed_images)

def display_samples(images, true_values, predicted_values, num_samples=9):
    # Choose random indices
    indices = np.random.choice(np.arange(images.shape[0]), size=num_samples, replace=False)

    # Initialize subplots
    fig, axs = plt.subplots(3, 3, figsize=(15, 15))

    # For each subplot
    for i, idx in enumerate(indices):
        # Get the sample image and its corresponding true and predicted values
        sample_image = images[idx]
        true_value = true_values[idx]
        predicted_value = predicted_values[idx]

        # Display the sample image with the true and predicted valence and arousal rates
        axs[i//3, i%3].imshow(sample_image, cmap='gray')
        axs[i//3, i%3].set_title(f'True: {true_value[0]:.2f}, {true_value[1]:.2f}\nPredicted: {predicted_value[0]:.2f}, {predicted_value[1]:.2f}')

    plt.tight_layout()
    plt.show()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from keras.preprocessing.image import load_img, img_to_array

# Load the images and their labels
image_dir = "./dataset/"
labels = []
images = []

emotion_to_valence_arousal = {
    'happy': (0.8, 0.6),
    'sad': (-0.8, -0.6),
    'neutral': (0.0, 0.0),
    'disgust': (-0.6, -0.6),
    'surprise': (0.6, 0.8),
    'angry': (-0.8, 0.8),
    'fear': (0.0, 0.8)
}

for folder in os.listdir(image_dir):
    for file in os.listdir(os.path.join(image_dir, folder)):
        if file.endswith(".jpg"):
            img = load_img(os.path.join(image_dir, folder, file), target_size=(48, 48))
            images.append(img_to_array(img))
            labels.append(emotion_to_valence_arousal[folder])  # Map the folder name to a valence-arousal pair

# Convert lists to numpy arrays
images = np.array(images) / 255.0  # Normalize the images
labels = np.array(labels)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.3, random_state=42)

# Feature Scaling
scaler = StandardScaler()
train_images_scaled = scaler.fit_transform(train_images.reshape(-1, train_images.shape[-1])).reshape(train_images.shape)
test_images_scaled = scaler.transform(test_images.reshape(-1, test_images.shape[-1])).reshape(test_images.shape)

# Flatten the images
train_images_flat = train_images_scaled.reshape(-1, train_images_scaled.shape[-1])
test_images_flat = test_images_scaled.reshape(-1, test_images_scaled.shape[-1])

# Apply Polynomial Features
poly = PolynomialFeatures(2)
train_images_poly = poly.fit_transform(train_images_flat)
test_images_poly = poly.transform(test_images_flat)

In [ ]:
print(f"Number of images: {len(images)}")
print(f"Number of labels: {len(labels)}")

print(f"Number of training images: {len(train_images)}")
print(f"Number of training labels: {len(train_labels)}")

In [ ]:
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.optimizers.legacy import Adam
from keras.layers import BatchNormalization
from keras.metrics import MeanSquaredError, MeanAbsoluteError
from keras.applications import VGG16
from sklearn.metrics import r2_score, explained_variance_score
from keras.regularizers import l2

datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,  
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

baseModel = VGG16(weights="imagenet", include_top=False, input_shape=(48, 48, 3))

headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
# Use a less aggressive regularization
headModel = Dense(512, activation="relu", kernel_regularizer=l2(0.005))(headModel)
headModel = BatchNormalization()(headModel)
# Use a lower dropout rate
headModel = Dropout(0.5)(headModel)
# Use a less aggressive regularization
headModel = Dense(512, activation="relu", kernel_regularizer=l2(0.005))(headModel)
headModel = BatchNormalization()(headModel)
# Use a lower dropout rate
headModel = Dropout(0.5)(headModel)
# Use a less aggressive regularization
headModel = Dense(1024, activation="relu", kernel_regularizer=l2(0.005))(headModel)
headModel = BatchNormalization()(headModel)
# Use a lower dropout rate
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="linear")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers[:-4]:
    layer.trainable = False
for layer in baseModel.layers[-4:]:
    layer.trainable = True
# Unfreeze more layers
for layer in baseModel.layers[:-8]:
    layer.trainable = False
for layer in baseModel.layers[-8:]:
    layer.trainable = True

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 40:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    print('Learning rate: ', lr)
    return lr

opt = Adam(lr=1e-3)
model.compile(optimizer=opt, 
              loss='mean_squared_error',  
              metrics=[MeanSquaredError(name='mse'), 
                       MeanAbsoluteError(name='mae')])

model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import CSVLogger, TensorBoard, EarlyStopping, LearningRateScheduler
from livelossplot.inputs.keras import PlotLossesCallback

batch_size = 32
train_generator = datagen.flow(train_images, train_labels, batch_size=batch_size)
val_generator = datagen.flow(test_images, test_labels, batch_size=batch_size)

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_loss', save_weights_only=True, mode='min', verbose=1)

csv_logger = CSVLogger('training.log')
tensorboard = TensorBoard(log_dir='./result', histogram_freq=0, write_graph=True, write_images=False)

# Create a new instance of the PlotLossesKeras callback
plot_losses = PlotLossesCallback()

# Create a LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Add it to your list of callbacks
callbacks = [plot_losses, checkpoint, reduce_lr, csv_logger, tensorboard, early_stopping]

try:
    model.fit(train_generator,
              steps_per_epoch=len(train_images) // batch_size,
              epochs=150,
              validation_data=val_generator,
              validation_steps=len(test_images) // batch_size,
              verbose=1, workers=4,
              callbacks=callbacks)
except KeyboardInterrupt:
    print("Training interrupted. Saving model...")
    model.save("model_weights.h5")

# Predict the valence-arousal coordinates for the test images
predictions = model.predict(test_images)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score

# Additional evaluation metrics
r2 = r2_score(test_labels, predictions)
evs = explained_variance_score(test_labels, predictions)  # Corrected line

print(f'R2 Score: {r2:.4f}')
print(f'Explained Variance Score: {evs:.4f}')

display_samples(test_images, test_labels, predictions)

# Predict the valence-arousal coordinates for the test images
predictions = model.predict(test_images)

# Plot the predicted valence-arousal coordinates
plt.scatter(predictions[:, 0], predictions[:, 1])
plt.xlabel('Valence')
plt.ylabel('Arousal')
plt.show()

In [ ]:
# Save the model architecture to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Save model's weights
model.save_weights("model_weights.h5")

# Save entire model
model.save("model.h5")